<a href="https://colab.research.google.com/github/revyellans/UAPML/blob/main/UAPML_(DistillBert)_revy_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

from transformers import (
    DistilBertTokenizerFast,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments
)

from torch.utils.data import Dataset, DataLoader

In [2]:
# ======================================================
# 1. LOAD DATA
# ======================================================

df = pd.read_csv("name_gender_dataset.csv")

# ======================================================
# 2. NORMALISASI KOLOM
# ======================================================

df.columns = df.columns.str.lower()
df = df.rename(columns={
    'nama': 'name',
    'jenis_kelamin': 'gender',
    'jk': 'gender'
})

df = df.dropna(subset=['name', 'gender'])
print("Data awal:", len(df))

Data awal: 147269


In [3]:
# ======================================================
# 3. PREPROCESSING NAMA (AMAN)
# ======================================================

def preprocess_name(text):
    text = str(text).lower().strip()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

df['name'] = df['name'].apply(preprocess_name)
df = df[df['name'].str.len() >= 2]

In [4]:
# ======================================================
# 4. NORMALISASI LABEL GENDER
# ======================================================

def normalize_gender(label):
    label = str(label).lower().strip()

    if (
        label.startswith('l') or
        label.startswith('m') or
        'male' in label or
        'pria' in label or
        'laki' in label
    ):
        return 'male'

    elif (
        label.startswith('p') or
        label.startswith('f') or
        'female' in label or
        'wanita' in label or
        'perempuan' in label
    ):
        return 'female'

    else:
        return None

df['gender'] = df['gender'].apply(normalize_gender)
df = df.dropna(subset=['gender'])

print("Data setelah preprocessing:", len(df))
print(df['gender'].value_counts())

Data setelah preprocessing: 147251
gender
female    89743
male      57508
Name: count, dtype: int64


In [5]:
# ======================================================
# 5. LABEL ENCODING
# ======================================================

le = LabelEncoder()
df['label'] = le.fit_transform(df['gender'])  # male/female → 0/1

# ======================================================
# 6. SPLIT DATA
# ======================================================

X_train, X_test, y_train, y_test = train_test_split(
    df['name'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

In [6]:
# ======================================================
# 7. TOKENIZER DISTILBERT (PRETRAINED)
# ======================================================

MODEL_NAME = "distilbert-base-uncased"
tokenizer = DistilBertTokenizerFast.from_pretrained(MODEL_NAME)

def tokenize(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=16,   # nama pendek
        return_tensors="pt"
    )

train_encodings = tokenize(X_train)
test_encodings = tokenize(X_test)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# ======================================================
# 8. DATASET TORCH
# ======================================================

class GenderDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = GenderDataset(train_encodings, y_train)
test_dataset = GenderDataset(test_encodings, y_test)

In [8]:
# ======================================================
# 9. LOAD MODEL DISTILBERT
# ======================================================

model = DistilBertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2
)

# ======================================================
# 10. FREEZE DISTILBERT ENCODER (SUPER CEPAT)
# ======================================================

for param in model.distilbert.parameters():
    param.requires_grad = False

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# ======================================================
# 11. TRAINING ARGUMENTS (FAST MODE)
# ======================================================

training_args = TrainingArguments(
    output_dir="./distilbert_output",
    eval_strategy="no",
    save_strategy="no",
    logging_strategy="steps",
    logging_steps=50,
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# ======================================================
# 12. TRAINER
# ======================================================

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.665900
100,0.650600
150,0.643500
200,0.628300
250,0.613100
300,0.604100
350,0.605900
400,0.593900
450,0.572700
500,0.570200


TrainOutput(global_step=7364, training_loss=0.5120559983250889, metrics={'train_runtime': 4706.8659, 'train_samples_per_second': 50.055, 'train_steps_per_second': 1.565, 'total_flos': 731468416953600.0, 'train_loss': 0.5120559983250889, 'epoch': 2.0})

In [10]:
# ======================================================
# 13. EVALUASI MANUAL (STABIL)
# ======================================================

model.eval()
test_loader = DataLoader(test_dataset, batch_size=32)

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"]
        )
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(batch["labels"].cpu().numpy())

print("Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=le.classes_))

Accuracy: 0.7758989507996333
              precision    recall  f1-score   support

      female       0.81      0.82      0.82     17949
        male       0.72      0.70      0.71     11502

    accuracy                           0.78     29451
   macro avg       0.76      0.76      0.76     29451
weighted avg       0.77      0.78      0.78     29451



In [15]:
# ======================================================
# 14. SIMPAN MODEL & TOKENIZER
# ======================================================

model.save_pretrained("gender_name_distilbert_model")
tokenizer.save_pretrained("gender_name_distilbert_model")

print("Model & tokenizer DistilBERT berhasil disimpan")

Model & tokenizer DistilBERT berhasil disimpan


In [14]:
# ======================================================
# 15. CONTOH PREDIKSI
# ======================================================

def predict_gender(name):
    name = preprocess_name(name)
    inputs = tokenizer(
        name,
        return_tensors="pt",
        truncation=True,
        max_length=16
    )
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return le.inverse_transform([pred])[0]

contoh = ["aisyah", "rizky", "revy", "iqmal"]

for n in contoh:
    print(n, "->", predict_gender(n))

aisyah -> female
rizky -> male
revy -> male
iqmal -> male
